In [1]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input
from keras.layers import Conv2D, MaxPool2D
from keras.models import Model
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
import weave

Using TensorFlow backend.


In [2]:
def lr_schedule(epoch):
    return (0.00100385 - 6.92308e-6*epoch)

def straight_stack(x, num_filters, rep = 3, weight_decay = 1e-4, dropout_start = 0.2, dropout_add = .1):
    for n in range(rep):
        x = Conv2D((2**n)*num_filters, (3,3), padding='same', activation='elu', kernel_regularizer=regularizers.l2(weight_decay))(x)
        x = BatchNormalization()(x)
        x = Conv2D((2**n)*num_filters, (3,3), padding='same', activation='elu', kernel_regularizer=regularizers.l2(weight_decay))(x)
        x = BatchNormalization()(x)
        x = MaxPool2D()(x)
        x = Dropout(dropout_start+n*dropout_add)(x)
    return x

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print(x_train.shape)
 
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

batch_size = 64

(50000, 3, 32, 32)


In [ ]:
viewer = keras.callbacks.TensorBoard(log_dir='./logs', 
                                     histogram_freq=0, 
                                     batch_size=batch_size, 
                                     write_graph=True)
num_filters = 4
num_val_test = 500
epochs = 2
model_details = {}
conv_layers = 3
for num_stack in [1,2,4]:
    inputs = Input(shape=(3,32,32))
    input_array = []
    pyrm_layers = int(np.log2(num_stack))
    for stack in range(num_stack):
        x = straight_stack(inputs, num_filters/num_stack, rep = conv_layers - pyrm_layers)
        input_array.append(x)
    if pyrm_layers > 0:
        x = weave.pyrm_net(input_size=(4,4,128),
                           n_layers = pyrm_layers,
                           n_filters_start = 128,
                           n_gpus = 1,
                           inputs = input_array,
                           r_filter = 1,
                           r_combine= 1,
                           min_dim = 2,
                           max_pool_loc = 5,
                           pure_combine = False,
                           end_max_pool = False)
    else:
        pass
    
    x = Flatten()(x)
    
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=[inputs], outputs=predictions)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.summary()
    num_params = model.count_params()
    
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test[0:num_val_test],y_test[0:num_val_test]),
                    callbacks=[LearningRateScheduler(lr_schedule), viewer])
    model_details[num_stack] = (num_params, model.evaluate(x_test,y_test))
    print(model_details)
    
    
    

Epoch 1/2
781/781 [==============================] - 436s 558ms/step - loss: 2.0201 - acc: 0.2863 - val_loss: 1.6055 - val_acc: 0.3960
Epoch 2/2
10000/10000 [==============================] - 50s 5ms/step
{1: (7646, [1.4586845079421997, 0.4645])}
GPU settings allow for inf layers
Minimum output size allow for 5 layers
Number of layers 1
First Layer Size: 1 (Number of Units)
('Disjoint:', True)
number of units 1
number of devices 2
number of inputs 2
First Layer Output Size: 1 (Number of Tensors)
Final Layer Size 1
Epoch 1/2
 54/781 [=>............................] - ETA: 52:10 - loss: 1.9839 - acc: 0.2705

In [ ]:
model_details
